In [6]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# #get admin key from s3
# my_env = 'fourfront-webdev'
# my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)


# ATAC RUNS
set_url = '/search/?experiments_in_set.experiment_type=ATAC-seq&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)
counter = 0
completed = 0
completed_acc = []

run_sets = [i for i in all_sets if "CHIP_Pipeline_1.1.1"  not in i.get('completed_processes', [])]
print(len(all_sets), 'total number of sets')
print(len(all_sets)-len(run_sets), 'sets completed')
print(len(run_sets), 'ready for processing')

11 total number of sets
0 sets completed
11 ready for processing


In [7]:
run_wfr = True
add_pc = False
add_tag = False

for a_set in run_sets: 
    print()
    print(a_set['accession'], end = " ")
    counter += 1
    # some feature to extract from each set
    control = ""  # True or False (True if set in scope is control)
    control_set = ""  # None (if no control exp is set), or the control experiment for the one in scope
    target_type = "" # Histone or TF (or None for control)
    paired = "" # single or paired
    organism = ""
    
    # pass attributions to new objects
    attributions = None
    
    replicate_exps = a_set['replicate_exps']
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])

    # get organism, target and control from the first replicate
    f_exp = replicate_exps[0]['replicate_exp']['uuid']
    f_exp_resp = ff_utils.get_metadata(f_exp, key = my_auth)
    biosample = f_exp_resp['biosample']
    organism = list(set([bs['individual']['organism']['name'] for bs in biosample['biosource']]))[0]
    print(organism)
    
    if organism not in ['mouse', 'human']:
        print('orgamism not ready', organism)
        continue

    attributions = get_attribution(ff_utils.get_metadata(f_exp_resp['files'][0]['uuid'], key = my_auth))

    ta = []
    # check for step1, and start if missing
    step1_status = 'Done'
    for an_exp in replicate_exps:
        exp_id = an_exp['replicate_exp']['accession']
        exp_resp = ff_utils.get_metadata(exp_id, my_auth)
        run_name = exp_resp['accession']
        print(run_name, end = ' status: ')
        exp_files, exp_obj, paired = get_chip_files(exp_resp, my_auth)

        step1_result = get_wfr_out_file(exp_files[0][0], 'encode-atacseq-aln 1.1.1', my_auth) 
        print(step1_result['status'])
        if step1_result['status'] == 'complete':
            ta.append(step1_result['atac.first_ta'])
            if add_pc:
                add_chip_processed_files_exp([step1_result['atac.first_ta']], my_auth)
        elif step1_result['status'] == 'running':
            step1_status = 'Incomplete'
        else:
            step1_status = 'Incomplete'
            if len(exp_files) > 2:
                print('WARNING More then 2 seq reps in exp')
            if run_wfr:
                print('starting run')
                run_missing_atac1(step_settings('encode-atacseq-aln', organism, attributions), 
                                  organism, paired, [exp_files], [exp_obj], my_env, my_auth, run_name)


    if step1_status != 'Done':
        continue
        
    
    print('set ready for part2')
    if len(ta) > 2:
        print('Experiment has more then 2 replicates, skipping')
        continue
    
    step2_result = get_wfr_out_file(ta[0], 'encode-atacseq-postaln 1.1.1', my_auth)
    if step2_result['status'] == 'complete':
        print('step2 is completed')
        if add_pc:
            # add_chip_processed_files_exp([]], my_auth)
            pass
        if add_tag:
            pass   
    elif step2_result['status'] == 'running':
        print('step2 is still running')
    else:
        step2_status = 'Incomplete' 
        if run_wfr:
            print('starting run')
            run_missing_atac2(step_settings('encode-atacseq-postaln', organism, attributions), 
                              organism, paired, ta, my_env, my_auth, a_set['accession'])
    
    

        


4DNESQZYEJ8N mouse
4DNEXBTUHDKP status: complete
4DNEX9NQ11PC status: complete
set ready for part2
step2 is completed

4DNESMBA9T3L human
4DNEXF6RQQ27 status: complete
4DNEXQUCGLW9 status: complete
4DNEXJGIUGRH status: complete
4DNEXBYHX8E4 status: complete
4DNEX97A5KCP status: complete
4DNEX6C42Q5B status: complete
4DNEXN7N1U82 status: complete
4DNEXMHGYBOJ status: complete
set ready for part2
Experiment has more then 2 replicates, skipping

4DNESJX5TSX7 mouse
4DNEXTWVMZCU status: complete
set ready for part2
step2 is completed

4DNESD3Q8OZ2 mouse
4DNEXMHN1BNA status: complete
set ready for part2
step2 is completed

4DNESW7PWWH8 mouse
4DNEXYD2QQAH status: running

4DNESWI45DYJ mouse
4DNEXFM1UJA4 status: running

4DNESDG4HNP9 human
4DNEXA7ALE6L status: complete
4DNEX3XJJPN6 status: complete
set ready for part2
step2 is completed

4DNESC9AQ3MU human
4DNEXLLQK7GM status: complete
4DNEXIX89R4K status: complete
set ready for part2
step2 is completed

4DNES2LXO15V human
4DNEXT76YJHV status

In [ ]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

recipe_no = 1
exp_type, step3 = recipe[recipe_no]
action = False
move_title = 'HiC Processing Pipeline - Preliminary Files'

set_url = '/search/?'+ \
          '&'.join(['experiments_in_set.experiment_type='+i for i in exp_type])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=released&status=released%20to%20project'

print(set_url)
set_url = '/search/?award.project=4DN&experimentset_type=replicate&lab.display_title=Chuck+Murry%2C+UW&status=pre-release&type=ExperimentSetReplicate'

# exp
# set_url = '/search/?'+ \
#           '&'.join(['experiment_type='+i for i in exp_type])+ \
#           '&type=Experiment&limit=all' + \
#           '&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)

ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_0.2.5" in i.get('completed_processes', [])]
print(len(ready_sets_1))
ready_sets_2 = []
for a_set in ready_sets_1:
    if a_set.get('other_processed_files'):
        print('a')
        print(a_set['accession'])
        if move_title in [i['title'] for i in a_set['other_processed_files']]:
            print('b')
            if a_set.get('processed_files'):
                print('c')
                print('WARN' ,a_set['accession'], 'has items in processed files, skipping ')
                continue
            else:
                ready_sets_2.append(a_set)
print(len(ready_sets_2), 'items are ready')

In [ ]:
# move other processed files to processed files field
action = True
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        if opc:
            print('There are files in processed_files field, expected empty', resp['accession'])
            return False
        else:
            print('it is possible that move already happened, no opc but pc', resp['accession'])
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'HiC Processing Pipeline - Preliminary Files'

print(len(ready_sets_2), 'experiment sets in scope')
for a_set in ready_sets_2:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_auth, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_auth)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_auth, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_auth)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)